In [5]:
import os.path
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling

TIFF = 'omuti.tif'

## Reproject a TIFF file to another Coordinate Reference System

For most ML tasks, we'll be dealing with raster images, without concerning ourselves with where in the physical world these images map to. At some point, if we wish to overlay these raster images to a real map that we can pan/zoom, we *may* need to reproject these raster images to an arbitrary CRS (though most libraries should allow us to specify the assumed CRS of the tif and do the reprojection/distortion/interpolation on the fly).

In [6]:
# Destination CRS
DST_CRS = 'EPSG:4326'

A function to reproject an input tif with a crs and save as an output tif follows.

In [7]:
def reproject_tif(input_tif, output_file, dst_crs):
    with rasterio.open(input_tif) as src:
        transform, width, height = calculate_default_transform(
            src.crs, dst_crs, src.width, src.height, *src.bounds)
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': dst_crs,
            'transform': transform,
            'width': width,
            'height': height
        })
        if output_file.endswith('.png'):
            kwargs['driver'] = 'PNG'

        with rasterio.open(output_file, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                destination, dst_transform = reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=dst_crs,
                    resampling=Resampling.nearest)

In [8]:
output_file = os.path.splitext(os.path.basename(TIFF))[0] + "_" + DST_CRS.replace(":", "_").lower() + ".png"
reproject_tif(TIFF, output_file, DST_CRS)

with rasterio.open(TIFF, 'r') as input_raster:
    print('CRS of the Input tif is {}'.format(input_raster.crs))
    
with rasterio.open(output_file, 'r') as output_raster:
    print('CRS of the reprojected tif is {}'.format(output_raster.crs))

CRS of the Input tif is EPSG:32733
CRS of the reprojected tif is EPSG:4326
